In [ ]:
# Goal of the Notebook

#This notebook contains a working prototype that uses voilà and jupyter widgets
#to provide a front-end to the application.

In [1]:
%load_ext autoreload

In [2]:
import pandas as pd
import dask.dataframe as dd
import tarfile
import scipy
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import surprise
import pickle 
import seaborn as sns
import ipywidgets as widgets
import os
import plotly.express as px
import datetime
import time

from surprise import Dataset, Reader
from surprise import SVD, SVDpp, NMF
from surprise.model_selection import train_test_split, GridSearchCV, cross_validate, KFold
from surprise.prediction_algorithms.knns import KNNBaseline
from surprise.prediction_algorithms.co_clustering import CoClustering
from surprise import accuracy

from sklearn.manifold import TSNE
from sklearn.neighbors import DistanceMetric
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors

from ipywidgets import interactive, widgets, interact, Layout
from IPython.core.display import HTML, display

from group_utils import *
from content_recommender import *
from collaborative_recommender import *
from heuristic_recommender import *

In [3]:
# Load data

In [4]:
famous_tracks            = pd.read_csv('data/features.csv')
df_1kfamous              = pd.read_csv('data/df_1kfamous.csv')
df_1kfamous_with_ratings = pd.read_pickle('data/df_1kfamous_with_ratings.pkl')
df_reduced               = pd.read_pickle('data/df_reduced.pkl')
famous_tracks_c          = pd.read_csv('data/tracks_clustered.csv')

In [5]:
# Print header for voilà

In [6]:
HTML("""
<div class="header">
  <img src="data/bewy.png"/>
  <p style="color:#4b4a98; text-align:center">Select sample users, pick a recommendation flavour, and enjoy !</p>
</div>

<style>
.header {
  padding: 20px;
  text-align: center;
  background: #e9edee;
  color: white;
  font-size: 30px;
}

</style>
""")

In [7]:
pref_artists = lambda x : set(x.sort_values(by='plays', ascending=False).head(10)['artist-name'])
values = df_1kfamous_with_ratings.groupby('catuser').apply(pref_artists)

In [8]:
user_selector = widgets.SelectMultiple(
    options=values,
    description="",
    disabled=False,
    layout=Layout(width='100%', height='200px', overflow='hidden')
)

In [9]:
file = open("data/loading.gif", "rb")
image = file.read()
loading = widgets.Image(
    value=image,
    format='png',
    width=500,
)

box_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='100%')

img_box = widgets.HBox(children=[loading],layout=box_layout)

In [10]:
HTML("""
<style>
div.ui-slider-range.ui-corner-all.ui-widget-header{
    background:red;
    height:0;
    margin-top:0;
}
</style>
""")

In [11]:
percentage = widgets.FloatRangeSlider(
    value=[0.7, 0.85],
    min=0,
    max=1.0,
    step=0.05,
    description='Percentage',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

In [12]:
gammas = widgets.FloatSlider(
    value=0.95,
    min=0.5,
    max=1.0,
    step=0.01,
    description='Gamma',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

In [13]:
%autoreload 2

In [14]:
def display_playlist(playlist):
    todisp = playlist.to_html(index=False).replace('class="dataframe"', 'class="dataframe" style="width:100%"')
    display(HTML(todisp))

out_loader = widgets.Output()

def display_from_users(gamma,perc, users):
    
    TOTAL_LENGTH = 20
    
    out_loader.clear_output()
    
    if len(users) > 1:
        with out_loader:
            user_ids = [values[values == user].index[0] for user in users]

            r1 = CollabRecommender('data/implicit_model.pkl', user_ids, famous_tracks_c, df_1kfamous_with_ratings, gamma=gamma)
            r2 = ContentRecommender(user_ids, famous_tracks_c, df_1kfamous_with_ratings)
            r3 = HeuristicRecommender(user_ids, famous_tracks_c, df_1kfamous_with_ratings, 'data/genre_artists.pkl')

            display(img_box)
            
            time.sleep(2)
            
            n1 = int(perc[0]*TOTAL_LENGTH)
            n2 = int((perc[1]-perc[0])*TOTAL_LENGTH)
            n3 = TOTAL_LENGTH - (n1+n2)
            
            lists = [r.compute_playlist(n) for n,r in zip([n1,n2,n3],[r1,r2,r3])]
            playlist = pd.concat(lists)
            out_loader.clear_output()
            
            for l in lists:
                display_playlist(l)
        
        
_ = interact(display_from_users, users=user_selector, perc=percentage, gamma=gammas)
display(out_loader)

interactive(children=(FloatSlider(value=0.95, continuous_update=False, description='Gamma', max=1.0, min=0.5, …

Output()